In [1]:
import subprocess

In [2]:
import json

In [3]:
import re

In [4]:
from neo4j import GraphDatabase

# Neo4j setup
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="gf6xb4kgeZKSS8p"

## Generate a response by querying the whole graph database

In [5]:
# Initialize Neo4j driver
neo4j_driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Function to retrieve data from Neo4j
def get_neo4j_data(query, parameters=None):
    with neo4j_driver.session() as session:
        result = session.run(query, parameters)
        return [record.data() for record in result]

# Define your Neo4j query
neo4j_query = """
MATCH (n)-[r]->(m)
RETURN n.id AS source_id, n.type AS source_type, type(r) AS relationship, m.id AS target_id, m.type AS target_type
LIMIT 400
"""

# Retrieve data from Neo4j
neo4j_data = get_neo4j_data(neo4j_query)

neo4j_context = "\n".join([
    f"{record['source_id']} ({record['source_type']}) -[{record['relationship']}]-> {record['target_id']} ({record['target_type']})"
    for record in neo4j_data
])

In [6]:
def escape_special_characters(text):
    # Escape single quotes and handle parentheses as needed for shell compatibility
    text = text.replace("'", "\\'")
    text = re.sub(r"[()]", "", text)
    return text

In [7]:
def get_answer_from_llama(neo4j_context, user_query):
    # Escape special characters in the graph data and query
    safe_neo4j_context = escape_special_characters(neo4j_context)
    safe_user_query = escape_special_characters(user_query)
    
    # Create the combined prompt
    full_prompt = f"Using the following graph data:\n{safe_neo4j_context}\n\nAnswer the query: '{safe_user_query}'"
    
    # Run the command and capture output
    command = f"echo \"{full_prompt}\" | ollama run llama3.2"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    
    # Check for any errors
    if result.stderr:
        print("Error:", result.stderr)
    else:
        print("Output:", result.stdout)
    
    return result.stdout.strip()

In [8]:
user_query = "Summarize the knowledge graph."
answer = get_answer_from_llama(neo4j_context, user_query)
print(answer)

Error: ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠦ ⠧ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠦ ⠇ ⠇ ⠋ ⠋ ⠹ ⠸ ⠸ ⠴ ⠦ ⠦ ⠇ ⠇ ⠋ ⠋ ⠹ ⠹ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠼ ⠴ ⠦ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠏ ⠙ ⠹ ⠸ ⠼ ⠼ ⠦ ⠧ ⠇ ⠇ ⠋ ⠙ ⠙ ⠸ ⠼ ⠴ ⠴ ⠧ ⠇ ⠇ ⠋ ⠙ ⠹ ⠹ ⠸ ⠴ 
Here is a summary of the knowledge graph:

**Cancer Research and Treatment**

* Cancer patients are treated with various therapies, including immunotherapy (e.g., PD-1/PD-L1 inhibitors)
* Cytokines (e.g., IL-5, IL-6) play a role in cancer treatment and may be associated with adverse events
* High-dose chemotherapy is used to treat lymphoma

**Immunology and Inflammation**

* Autoimmune disorders may prevent immune checkpoint inhibition
* T helper cells occur in patients treated with certain drugs
* Immune-related adverse events (IRAEs) are a concern for cancer patients treated with PD-1/PD-L1 inhibitors
* Antioxidant molecules may prevent cellular senescence and reactive oxygen species

**Targeted Therapies**

* Axitinib is used to treat renal cell carcinoma and glioblastoma, and induces cellular senescence and

In [9]:
user_query = "What is the use of trastuzumab deruxtecan in treating cancer inside knowledge graph"
answer = get_answer_from_llama(neo4j_context, user_query)
print(answer)

Error: ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠸ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠴ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠧ ⠇ ⠋ ⠙ ⠹ ⠹ ⠼ ⠴ ⠴ ⠧ ⠇ ⠏ ⠏ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠧ ⠏ ⠋ ⠙ ⠹ ⠹ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠏ ⠙ ⠙ ⠸ ⠼ ⠴ ⠴ ⠧ 
The query "What is the use of trastuzumab deruxtecan in treating cancer inside knowledge graph" can be answered by searching for relevant information within the knowledge graph.

Here is a possible answer based on the provided knowledge:

Trastuzumab Deruxtecan is used to treat HER2-positive breast cancer. It is a monoclonal antibody that targets the HER2 protein, which is overexpressed in some breast cancer cells.

According to the knowledge graph, trastuzumab deruxtecan is used in combination with chemotherapy and/or other treatments for:

* Treatment of locally advanced or metastatic HER2-positive breast cancer
* Treatment of gastric cancer
* Treatment of triple-negative breast cancer

It is also mentioned that trastuzumab deruxtecan has been shown to have anti-tumor activity against tumors expressing th